In [0]:
!pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 5.1MB/s 
     |████████████████████████████████| 675kB 50.4MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609187 sha256=93ff2c29ad64c3d161794f5e2f71ef306307cf8a85634e8c92c72168c31dde45
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained mohousedel tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [0]:
from google.colab import files
uploaded = files.upload()


Saving test.tsv to test.tsv


In [0]:
import pandas as pd
train=pd.read_csv("train.tsv","\t", header=None, names=["is_duplicate","question 1","question 2","id"])
train = train.dropna()

In [0]:
import pandas as pd
dev=pd.read_csv("test.tsv","\t", header=None, names=["is_duplicate","question 1","question 2","id"])
dev = dev.dropna()

In [0]:
from nltk import sent_tokenize
import math
examples_sentence1_dev = [] 
examples_sentence2_dev = [] 
labelsd = []
for i in range(len(dev["question 1"])):
    data = sent_tokenize(dev["question 1"].iloc[i])
    data1 = sent_tokenize(dev["question 2"].iloc[i])
    x="[CLS] "
    y="[CLS] "
    for k in data:
        x=  x+k+" [SEP]"
        examples_sentence1_dev.append(x)   
    for j in data1:
        y = y+j+" [SEP]"        
        examples_sentence2_dev.append(y)   
    labelsd.append(dev["is_duplicate"].iloc[i])

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def get_embedding(each_example):
  tokenized_text = tokenizer.tokenize(each_example) 
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids=[]tokenized_texttokenized_texttokenized_texttokenitokenized_texttokenized_texttokenized_textzed_text
  x = tokenized_text.copy()
  t=0
  for i in range(tokenized_text.count('[SEP]')):
    index = x.index('[SEP]')
    if t%2 == 0:
      segments_ids.extend([0] * (index+1))
    else:
      segments_ids.extend([1] * (index+1))
    x=x[index+1:]
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model.eval()
  with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor.cuda(), segments_tensors.cuda())
  token_embeddings = torch.stack(encoded_layers, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)
  token_vecs_sum = []
  for token in token_embeddings:
      sum_vec = torch.sum(token[-4:], dim=0)
      token_vecs_sum.append(sum_vec)
  token_vecs = encoded_layers[11][0]
  sentence_embedding = torch.mean(token_vecs, dim=0)
  return(token_vecs, sentence_embedding)

In [0]:
model = BertModel.from_pretrained('bert-base-uncased').cuda()

token_vec_sen1 = []
sen_emb1 = []
token_vec_sen2 = []
sen_emb2 = []
for each_example in range(365801,len(examples_sentence1_dev)):
  t,s =get_embedding(examples_sentence1_dev[each_example])
  token_vec_sen1.append(t)
  sen_emb1.append(s)
  t,s =get_embedding(examples_sentence2_dev[each_example])
  token_vec_sen2.append(t)
  sen_emb2.append(s)


# Load pre-trained model (weights)


# Put the model in "evaluation" mode, meaning feed-forward operation.



In [0]:
import pickle
with open("sen_emb1_test.pkl","wb") as g:
  pickle.dump(sen_emb1,g)

with open("sen_emb2_test.pkl","wb") as g:
  pickle.dump(sen_emb2,g)


In [0]:
import pickle
with open("token_vec_sen1_test.pkl","wb") as g:
  pickle.dump(token_vec_sen1,g)

with open("token_vec_sen2_test.pkl","wb") as g:
  pickle.dump(token_vec_sen2,g)


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': "1KLKt3u4NFJW-gDP4vpsX8dI4XNLvNHKB"}]})
file.SetContentFile('token_vec_sen2_test.pkl')
file.Upload() 


In [0]:
model = BertModel.from_pretrained('bert-base-uncased').cuda()
token_vec_sen1 = []
sen_emb1 = []
token_vec_sen2 = []
sen_emb2 = []

for each_example in range(len(examples_sentence1)):
  t,s =get_embedding(examples_sentence1[each_example])
  token_vec_sen1.append(t)
  sen_emb1.append(s)
  t,s =get_embedding(examples_sentence2[each_example])
  token_vec_sen2.append(t)
  sen_emb2.append(s)


In [0]:
294812-151734+151174+71548

365800

In [0]:
143078+151174+71548

365800

In [0]:
len(token_vec_sen1)

71548

In [0]:
len(x)+151174

294252

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
